In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_curve,auc
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import matplotlib.pyplot as plt

In [5]:
# Loading the Data
data = pd.read_csv("CustomerData.csv",header=0)

In [6]:
# understand the data
data.shape

(3209, 14)

In [7]:
# Display coulumns
data.columns

Index(['CustomerID', 'City', 'NoOfChildren', 'MinAgeOfChild', 'MaxAgeOfChild',
       'Tenure', 'FrquncyOfPurchase', 'NoOfUnitsPurchased', 'FrequencyOFPlay',
       'NoOfGamesPlayed', 'NoOfGamesBought', 'FavoriteChannelOfTransaction',
       'FavoriteGame', 'TotalRevenueGenerated'],
      dtype='object')

In [8]:
#Display index
data.index

RangeIndex(start=0, stop=3209, step=1)

In [9]:
# SEe the top rows of the data
data[:3]

CustomerID  City          ...            FavoriteGame  TotalRevenueGenerated
0        1001     1          ...                 Uniform                 107.51
1        1002     1          ...                 Uniform                 382.40
2        1003     1          ...                 Uniform                 135.01

[3 rows x 14 columns]

In [10]:
# summary Statistics of each column
data.describe()

CustomerID          ...            TotalRevenueGenerated
count  3209.000000          ...                      3209.000000
mean   2605.000000          ...                       168.477183
std     926.502833          ...                        81.798528
min    1001.000000          ...                       100.000000
25%    1803.000000          ...                       116.640000
50%    2605.000000          ...                       142.390000
75%    3407.000000          ...                       191.250000
max    4209.000000          ...                       990.560000

[8 rows x 12 columns]

In [11]:
# Display datatype of each variable
data.dtypes

CustomerID                        int64
City                              int64
NoOfChildren                      int64
MinAgeOfChild                     int64
MaxAgeOfChild                     int64
Tenure                            int64
FrquncyOfPurchase                 int64
NoOfUnitsPurchased                int64
FrequencyOFPlay                   int64
NoOfGamesPlayed                   int64
NoOfGamesBought                   int64
FavoriteChannelOfTransaction     object
FavoriteGame                     object
TotalRevenueGenerated           float64
dtype: object

In [ ]:
# Observations
#Observations
#City is Categorical but is interpreted as int64 
#FavoriteChannelOfTransaction,FavoriteGame   interpreted as Object but we should convert to Categorical

In [12]:
#Convert all the attributes to appropriate type
for col in ['City', 'FavoriteChannelOfTransaction','FavoriteGame']:
    data[col] = data[col].astype('category')

In [13]:
data.dtypes

CustomerID                         int64
City                            category
NoOfChildren                       int64
MinAgeOfChild                      int64
MaxAgeOfChild                      int64
Tenure                             int64
FrquncyOfPurchase                  int64
NoOfUnitsPurchased                 int64
FrequencyOFPlay                    int64
NoOfGamesPlayed                    int64
NoOfGamesBought                    int64
FavoriteChannelOfTransaction    category
FavoriteGame                    category
TotalRevenueGenerated            float64
dtype: object

In [15]:
#Delete CustomerID attribute
np.size(np.unique(data.CustomerID,return_counts=True)[0])

3209

In [16]:
data.drop('CustomerID',axis=1, inplace=True)

In [17]:
# Handling Missing data
data.isnull().sum()

City                            0
NoOfChildren                    0
MinAgeOfChild                   0
MaxAgeOfChild                   0
Tenure                          0
FrquncyOfPurchase               0
NoOfUnitsPurchased              0
FrequencyOFPlay                 0
NoOfGamesPlayed                 0
NoOfGamesBought                 0
FavoriteChannelOfTransaction    0
FavoriteGame                    0
TotalRevenueGenerated           0
dtype: int64

In [18]:
#Converting Categorical to Numeric
#For some of the models all the independent attribute should be of type numeric and Linear Regression model is one among them. But this data set has some categorial attributes.

#'pandas.get_dummies' To convert convert categorical variable into dummy/indicator variables
print(data.columns)

Index(['City', 'NoOfChildren', 'MinAgeOfChild', 'MaxAgeOfChild', 'Tenure',
       'FrquncyOfPurchase', 'NoOfUnitsPurchased', 'FrequencyOFPlay',
       'NoOfGamesPlayed', 'NoOfGamesBought', 'FavoriteChannelOfTransaction',
       'FavoriteGame', 'TotalRevenueGenerated'],
      dtype='object')


In [19]:
#Creating dummy variables.

#If we have k levels in a category, then we create k-1 dummy variables as the last one would be redundant. 
#So we use the parameter drop_first in pd.get_dummies function that drops the first level in each of the category

categorical_Attributes = data.select_dtypes(include=['category']).columns




In [20]:
data = pd.get_dummies(columns=categorical_Attributes,data=data,prefix=categorical_Attributes,prefix_sep="_",drop_first=True)

In [22]:
print(data.columns, data.shape)

Index(['NoOfChildren', 'MinAgeOfChild', 'MaxAgeOfChild', 'Tenure',
       'FrquncyOfPurchase', 'NoOfUnitsPurchased', 'FrequencyOFPlay',
       'NoOfGamesPlayed', 'NoOfGamesBought', 'TotalRevenueGenerated', 'City_2',
       'FavoriteChannelOfTransaction_Uniform', 'FavoriteGame_Uniform'],
      dtype='object') (3209, 13)


In [23]:
# Target Attribute Distribution
pd.value_counts(data['TotalRevenueGenerated'])

100.00    85
105.00    74
110.00    61
102.50    57
120.00    53
107.50    46
117.50    41
132.50    39
145.00    39
115.00    38
130.00    38
112.50    37
125.00    34
137.50    31
122.50    31
140.00    30
150.00    29
152.50    27
142.50    27
127.50    25
135.00    24
180.00    23
170.00    21
155.00    18
172.50    17
165.00    14
160.00    13
195.00    13
157.50    12
147.50    11
          ..
100.13     1
155.03     1
120.26     1
171.97     1
312.51     1
268.23     1
114.23     1
100.68     1
139.79     1
129.88     1
125.32     1
123.91     1
281.25     1
122.25     1
141.75     1
218.50     1
255.00     1
452.50     1
100.50     1
139.50     1
185.89     1
252.50     1
111.50     1
317.64     1
249.00     1
122.75     1
105.09     1
106.43     1
366.79     1
210.71     1
Name: TotalRevenueGenerated, Length: 1672, dtype: int64

In [ ]:
# Split the data in to train and test
#sklearn.model_selection.train_test_split

#Split arrays or matrices into random train and test subsets

In [25]:
##Performing train test split on the data
X,y = data.loc[:,data.columns!='TotalRevenueGenerated'].values, data.loc[:,'TotalRevenueGenerated'].values

In [36]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=123)

In [37]:
#To get the distribution in the target in train and test
print(pd.value_counts(y_train))
print(pd.value_counts(y_test))

100.00    66
105.00    51
110.00    42
120.00    36
102.50    35
107.50    32
117.50    30
112.50    28
115.00    25
145.00    25
132.50    24
125.00    24
137.50    22
140.00    22
122.50    22
130.00    21
152.50    21
127.50    20
150.00    18
180.00    18
135.00    15
142.50    15
170.00    13
195.00    11
172.50    11
157.50    11
155.00    11
165.00     9
175.00     7
207.50     7
          ..
284.53     1
105.04     1
126.13     1
119.23     1
112.59     1
270.93     1
151.74     1
338.19     1
162.81     1
143.70     1
102.48     1
100.94     1
103.41     1
284.32     1
159.67     1
116.11     1
154.12     1
103.87     1
101.32     1
106.71     1
269.25     1
425.00     1
220.50     1
170.59     1
126.75     1
167.00     1
290.00     1
123.51     1
114.50     1
104.67     1
Length: 1277, dtype: int64
105.00    23
102.50    22
100.00    19
110.00    19
120.00    17
130.00    17
132.50    15
107.50    14
145.00    14
115.00    13
142.50    12
150.00    11
117.50    11
125.00    1

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3209 entries, 0 to 3208
Data columns (total 13 columns):
NoOfChildren                            3209 non-null int64
MinAgeOfChild                           3209 non-null int64
MaxAgeOfChild                           3209 non-null int64
Tenure                                  3209 non-null int64
FrquncyOfPurchase                       3209 non-null int64
NoOfUnitsPurchased                      3209 non-null int64
FrequencyOFPlay                         3209 non-null int64
NoOfGamesPlayed                         3209 non-null int64
NoOfGamesBought                         3209 non-null int64
TotalRevenueGenerated                   3209 non-null float64
City_2                                  3209 non-null uint8
FavoriteChannelOfTransaction_Uniform    3209 non-null uint8
FavoriteGame_Uniform                    3209 non-null uint8
dtypes: float64(1), int64(9), uint8(3)
memory usage: 260.2 KB


In [32]:
# Perceptron Model Building
perceptron_model = Sequential()
perceptron_model.add(Dense(1, input_dim = 12,activation='linear',kernel_initializer='normal'))


In [40]:
perceptron_model.compile(loss='mse',optimizer='adam',metrics=['mse'])

In [41]:
perceptron_model.fit(X_train, y_train, epochs=30, batch_size=64)

Epoch 1/30
2246/2246 [==============================] - 3s 1ms/step - loss: 14958.1478 - mean_squared_error: 14958.1478
Epoch 2/30
2246/2246 [==============================] - 0s 124us/step - loss: 11407.0764 - mean_squared_error: 11407.0764
Epoch 3/30
2246/2246 [==============================] - 0s 131us/step - loss: 9842.7208 - mean_squared_error: 9842.7208
Epoch 4/30
2246/2246 [==============================] - 0s 117us/step - loss: 8727.4979 - mean_squared_error: 8727.4979
Epoch 5/30
2246/2246 [==============================] - 0s 118us/step - loss: 7962.3026 - mean_squared_error: 7962.3026
Epoch 6/30
2246/2246 [==============================] - 0s 120us/step - loss: 7413.6434 - mean_squared_error: 7413.6434
Epoch 7/30
2246/2246 [==============================] - 0s 119us/step - loss: 7040.5680 - mean_squared_error: 7040.5680
Epoch 8/30
2246/2246 [==============================] - 0s 121us/step - loss: 6769.6212 - mean_squared_error: 6769.62120s - loss: 5573.3110 - mean_squared_err

In [42]:
# Predictions
y_pred = perceptron_model.predict_classes(X_test)
y_pred_train = perceptron_model.predict_classes(X_train)

In [43]:
from sklearn import metrics

In [44]:
print("Mean absolute Error:", metrics.mean_absolute_error(y_pred,y_test))
print("Mean Squreared Error:", metrics.mean_squared_error(y_pred,y_test))
print("Root Mean Square Error:", np.sqrt(metrics.mean_squared_error(y_pred,y_test)))

Mean absolute Error: 166.14197300103842
Mean Squreared Error: 34630.911092211834
Root Mean Square Error: 186.09382335857316
